In [1]:
#import pytc
import ipywidgets as widgets
from IPython.display import display, clear_output
import glob

import interface

%matplotlib inline

SystemError: Parent module '' not loaded, cannot perform relative import